# Merging two Models

To merge two models in one model reflects the prove of concept, which shall show that the merged model is better than the two models on their own. Therefore, I have trained two models on different pictures with a small number of epochs. This results in two single models, which do not perform well.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd

import PIL.Image as Image
from IPython.display import Image, display
import matplotlib.cm as cm
import tensorflow as tf
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import tensorflow as tf
import numpy as np

## Create test dataset

To prove if the merged model is better than the models before I create a test dataset.

In [2]:
directory = "data/"

BATCH_SIZE = 32
IMG_SIZE = (192, 192)

validation_dataset = tf.keras.utils.image_dataset_from_directory(directory,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE,
                                                            shuffle=True,
                                                            seed=999,
                                                            validation_split=0.20,
                                                            subset = "validation")

val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches)

Found 824 files belonging to 2 classes.
Using 164 files for validation.


In [3]:
# Here the accuracy of model 1 is evaluated
loaded_model_1 = tf.keras.models.load_model('models/model1.keras')

loss, accuracy_model_1 = loaded_model_1.evaluate(test_dataset)
print('Test accuracy :', accuracy_model_1)

6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 159ms/step - accuracy: 0.8766 - loss: 0.4349
Test accuracy : 0.8780487775802612


In [4]:
# Here the accuracy of model 1 is evaluated
loaded_model_2 = tf.keras.models.load_model('models/model2.keras')

loss, accuracy_model_2 = loaded_model_2.evaluate(test_dataset)
print('Test accuracy :', accuracy_model_2)

6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 293ms/step - accuracy: 0.8059 - loss: 0.5314
Test accuracy : 0.8048780560493469


In [5]:
# This code merges the model. The mechanism which is used to achieve this is to average all the weight of the different models and cretae a new model.
# There is also 
def average_models(models):
    new_model = tf.keras.models.clone_model(models[0])
    new_weights = [np.mean([model.get_weights()[i] for model in models], axis=0) for i in range(len(models[0].get_weights()))]
    new_model.set_weights(new_weights)
    return new_model

models = [loaded_model_1, loaded_model_2]
average_model = average_models(models)

base_learning_rate = 0.0001

average_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

average_model.summary()

# And here the accuracy of the merged model is evaluated
loss, average_model_accuracy = average_model.evaluate(test_dataset)
print('Test accuracy :', average_model_accuracy)


Model: "functional_26"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_25 (InputLayer)     │ (None, 192, 192, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_8 (Sequential)       │ (None, 192, 192, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_192            │ (None, 6, 6, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_8      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 148ms/step - accuracy: 0.8939 - loss: 0.4700
Test accuracy : 0.9024389982223511


In [6]:
if average_model_accuracy > accuracy_model_1 and average_model_accuracy > accuracy_model_2:
    print("The merged Model is better")
else:
    print("The merged Model is not better")


The merged Model is better
